## 🌟 스토리: AI 스타트업 “유니코드(UNICODE)”의 하루  
당신은 국내 유망 AI 스타트업 **“유니코드”**의 주니어 엔지니어입니다.  
회사에서는 이제 막 출시한 뉴스 분류 챗봇과 시맨틱 검색 엔진을 안정화하고, SBERT 임베딩을 직접 파인튜닝해 보고자 합니다.  
오늘 당신에게 주어진 미션은 다음과 같습니다:  
1. **다중 라벨 뉴스 분류** (BERT)  
2. **Sentence-BERT 챗봇**  
3. **Faiss + SBERT 시맨틱 검색기**  
4. **SBERT 파인튜닝**  
각 파트별로 주어진 코드 스켈레톤을 완성하여, toy 데이터를 이용한 작은 프로젝트지만 **정상 동작하는 예제**를 완성해 주세요.

In [3]:
#!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 48.1 MB/s eta 0:00:00


In [4]:
import os
import random
import numpy as np
import torch
from datasets import Dataset, Features, Value, Sequence
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import BertTokenizerFast, BertForSequenceClassification, TrainingArguments, Trainer
from sentence_transformers import SentenceTransformer, models, losses
import faiss


SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


In [5]:
import torch

# Check if MPS is available
print("MPS available:", torch.backends.mps.is_available())
print("CUDA available:", torch.cuda.is_available())

# Set device
device = torch.device("mps") if torch.backends.mps.is_available() else \
         torch.device("cuda") if torch.cuda.is_available() else \
         torch.device("cpu")
print("Using device:", device)



MPS available: False
CUDA available: True
Using device: cuda


In [7]:
# ===== Part 1: 다중 라벨 뉴스 분류 =====
raw_samples = [
    ('삼성전자, 2분기 실적 발표… 영업이익 15% 증가', ['경제','테크']),
    ('尹 대통령, G7 회의 참석 차 히로시마 출국', ['정치']),
    ('LG 트윈스, 9회 끝내기 홈런으로 한화에 승리', ['스포츠']),
    ('AI 챗봇 서비스 출시로 IT 업계 경쟁 치열', ['테크']),
    ('한국, IMF 이후 최초로 국가 신용등급 상향', ['경제']),
    ('김연아, 아이스쇼에서 완벽한 연기 선보여', ['스포츠','연예']),
    ('넷플릭스, 한국 오리지널 콘텐츠 투자 확대', ['연예','경제']),
]

texts = [s for s, _ in raw_samples]
labels = [l for _, l in raw_samples]

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=sorted({t for lb in labels for t in lb}))
y = mlb.fit_transform(labels)

from datasets import Dataset
dataset = Dataset.from_dict({'text': texts, 'labels': list(y)})

# TODO 1: train/test split (test_size=0.3, seed=42)
dataset = dataset.train_test_split(test_size=0.3, seed=42)


    # TODO 2: example['labels'] 항목을 float 리스트로 변환
def cast_labels_to_float(example):
    example['labels'] = [float(v) for v in example['labels']]
    return example

# TODO 3: dataset.map(cast_labels_to_float)
dataset = dataset.map(cast_labels_to_float)

from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

def tokenize_batch(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=64)

# TODO 4: dataset.map(tokenize_batch, batched=True)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')
def tokenize_batch(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=64)

dataset = dataset.map(tokenize_batch, batched=True)
# TODO 5: dataset.set_format(type='torch', columns=['input_ids','attention_mask','labels'])
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [8]:
# ===== Part 2: 문장 임베딩 기반 간단 FAQ 챗봇 =====
faq_pairs = [
    ("환불이 가능한가요?", "상품 수령 후 7일 이내 환불 가능합니다."),
    ("배송 기간은 얼마나 걸리나요?", "통상 2~3일 소요됩니다."),
    ("AS는 어디로 문의하나요?", "고객센터(1234-5678)로 연락 주세요."),
]
questions, answers = zip(*faq_pairs)

from sentence_transformers import SentenceTransformer
sbert = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', device=device)
q_embeddings = sbert.encode(list(questions), convert_to_tensor=True, normalize_embeddings=True)

def chatbot(user_query: str, top_k: int = 1):
    query_emb = sbert.encode(user_query, convert_to_tensor=True, normalize_embeddings=True)
    scores = (q_embeddings @ query_emb).squeeze()
    best_idx = torch.argmax(scores).item()
    return answers[best_idx]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# ===== Part 3: Faiss 시맨틱 검색 =====
import faiss
from sentence_transformers import SentenceTransformer

corpus = [
    "파이썬은 배우기 쉬운 프로그래밍 언어입니다.",
    "BERT는 구글에서 발표한 사전학습 언어 모델입니다.",
    "Faiss는 페이스북 AI 리서치에서 개발한 벡터 검색 라이브러리입니다.",
    "Sentence-BERT는 문장 임베딩을 효율적으로 생성합니다.",
]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', device=device)
corpus_emb = model.encode(corpus, convert_to_numpy=True, normalize_embeddings=True)

index = faiss.IndexFlatIP(corpus_emb.shape[1])
index.add(corpus_emb)

def semantic_search(query: str, k: int = 3):
    q_vec = model.encode(query, convert_to_numpy=True, normalize_embeddings=True).reshape(1, -1)
    scores, idxs = index.search(q_vec, k)
    # TODO: (문장, 점수) 튜플로 된 리스트 형태로 반환
    return [(corpus[i], float(scores[0][j])) for j, i in enumerate(idxs[0])]

In [12]:
# ===== Part 4: SBERT 파인튜닝 =====
from sentence_transformers import SentenceTransformer, models, losses
from torch.utils.data import DataLoader

import wandb
wandb.init(anonymous="allow")

train_s1 = ["오늘 날씨 어때?", "주문 취소할래요", "반품 가능한가요?"]
train_s2 = ["지금 날씨 알려줘", "주문을 취소하고 싶습니다", "상품을 반품하고 싶어요"]
labels = [1.0, 1.0, 1.0]

word_emb = models.Transformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
pool    = models.Pooling(word_emb.get_word_embedding_dimension(), 'mean')
sbert_train = SentenceTransformer(modules=[word_emb, pool], device=device)

from sentence_transformers import InputExample, losses as st_losses
train_examples = [InputExample(texts=[a, b], label=l)
                  for a, b, l in zip(train_s1, train_s2, labels)]
train_dataloader = DataLoader(train_examples, batch_size=2, shuffle=True)
train_loss       = st_losses.CosineSimilarityLoss(sbert_train)

# TODO: model.fit()을 사용해 1 epoch 파인튜닝 (warmup_steps=10)
# SBERT 파인튜닝
sbert_train.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    warmup_steps=10,
    show_progress_bar=True
)
print(sbert_train.encode("제품 환불하고 싶어요")[:5], "...")

train/epoch,▁
train/global_step,▁
total_flos,0
train/epoch,1
train/global_step,2
train_loss,0.21486
train_runtime,0.7247
train_samples_per_second,4.139
train_steps_per_second,2.76


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


[ 0.09154709  0.19182989  0.0221106  -0.0387358  -0.2367134 ] ...


In [13]:
# 몇번째가 가장 알맞는지 확인하는 코드
from numpy import dot
from numpy.linalg import norm
import numpy as np

# 사용자 쿼리
query = "제품 환불하고 싶어요"

# SBERT 임베딩
query_vec = sbert_train.encode(query, convert_to_numpy=True, normalize_embeddings=True)
s1_vecs   = sbert_train.encode(train_s1, convert_to_numpy=True, normalize_embeddings=True)
s2_vecs   = sbert_train.encode(train_s2, convert_to_numpy=True, normalize_embeddings=True)

# 각 쌍에 대해 유사도 계산 (train_s1 vs 쿼리, train_s2 vs 쿼리 → 평균 유사도)
similarities = []
for i in range(len(train_s1)):
    sim1 = dot(query_vec, s1_vecs[i])
    sim2 = dot(query_vec, s2_vecs[i])
    avg_sim = (sim1 + sim2) / 2
    similarities.append(avg_sim)

# 가장 유사한 인덱스 찾기
best_idx = int(np.argmax(similarities))
best_score = similarities[best_idx]

# 결과 출력
print(f"쿼리: '{query}'")
print(f"가장 유사한 항목 index: {best_idx}")
print(f"train_s1: '{train_s1[best_idx]}'")
print(f"train_s2: '{train_s2[best_idx]}'")
print(f"평균 유사도: {best_score:.4f}")


쿼리: '제품 환불하고 싶어요'
가장 유사한 항목 index: 1
train_s1: '주문 취소할래요'
train_s2: '주문을 취소하고 싶습니다'
평균 유사도: 0.9498


✔️ **제출 전 체크리스트**

- 각 Part의 `TODO`를 모두 완성했나요?  
- `device` 설정(cuda/mps/cpu) 확인했나요?  
- 예제 함수(`predict_multilabel`, `chatbot`, `semantic_search`)가 정상 동작하는지 확인했나요?  
- 결과 출력 예시 스크린샷을 함께 제출하세요.

즐겁게 코딩하시고, 오류가 있으면 언제든 질문해 주세요!